In [4]:
from DataHandler import DataAcquisitionHandler

In [5]:
handler = DataAcquisitionHandler()
handler.load('data.pkl')
print(handler.get_data())

{'box_data': {'metadata': {'start_time': 1698254096.9270284, 'length': 66.15101170539856, 'trials': [{'timestamp': (1.6124939918518066, 2.6129705905914307), 'label': False}, {'timestamp': (4.535520553588867, 5.535949230194092), 'label': False}, {'timestamp': (7.843360424041748, 8.843704223632812), 'label': False}, {'timestamp': (11.05046534538269, 12.050799369812012), 'label': False}, {'timestamp': (13.656383991241455, 14.65677547454834), 'label': False}, {'timestamp': (16.96614170074463, 17.966572046279907), 'label': False}, {'timestamp': (20.37415909767151, 21.384049654006958), 'label': True}, {'timestamp': (23.191528797149658, 24.191861152648926), 'label': False}, {'timestamp': (26.600045442581177, 27.60047149658203), 'label': False}, {'timestamp': (30.00849151611328, 31.008914470672607), 'label': False}, {'timestamp': (33.31735062599182, 34.318182706832886), 'label': False}, {'timestamp': (36.024909019470215, 37.03983163833618), 'label': True}, {'timestamp': (39.247034788131714, 40

In [ ]:
# TODO: make the data into a seperate object so you can do things with it more easily w/o navigating the labyrinth of a dictionary